In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from pyspark.sql import SparkSession
from preprocessing.Preprocessor import Preprocessor
from pyspark.ml.classification import LogisticRegression
from metrics.metrics import Metrics
from pyspark.ml.tuning import TrainValidationSplit, CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [4]:
pp = Preprocessor(spark)

In [5]:
train = pp.to_tfidf('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
train = train.sample(False,0.6)
test =  pp.to_tfidf('Cleaned Data/test.csv')
train.cache()
test.cache()

DataFrame[features: vector, label: int]

In [6]:
lr = LogisticRegression(maxIter=100, aggregationDepth = 2, elasticNetParam=0)

In [7]:
"""%%time
model = lr.fit(train)"""

CPU times: user 12.6 ms, sys: 3.36 ms, total: 16 ms
Wall time: 9.72 s


In [8]:
test_m = Metrics()
test = model.transform(test).select('prediction', 'label')
test_m.f1(test)
test_m.confusion_matrix(test)

F1 score for given DataFrame: 75.1

                Predictions
            negative    neutral     positive    
negative    2686        146         1299        
neutral     403         116         960         
positive    659         190         9934        
(True labels are on the side)



In [9]:
"""encoding_lst = [pp.to_tfidf, pp.to_bow]
#pp.to_bow --> include in encoding_lst

for i in encoding_lst:
    train = i('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
    test =  i('Cleaned Data/test.csv')
    #train = pp.balance_classes(train)
    train.cache()
    test.cache()
    lr = LogisticRegression(maxIter=100, aggregationDepth = 2, elasticNetParam=0)
    model = lr.fit(train)
    test_m = Metrics()
    test = model.transform(test).select('prediction', 'label')
    #y = str(i)
    #x = str((test_m.f1(test)))
    #print(y + x, end = "")
    test_m.f1(test)
    test_m.confusion_matrix(test)"""

'encoding_lst = [pp.to_tfidf, pp.to_bow]\n#pp.to_bow --> include in encoding_lst\n\nfor i in encoding_lst:\n    train = i(\'Cleaned Data/train.csv\', min_frequency=275., remove_stop_words = False)\n    test =  i(\'Cleaned Data/test.csv\')\n    #train = pp.balance_classes(train)\n    train.cache()\n    test.cache()\n    lr = LogisticRegression(maxIter=100, aggregationDepth = 2, elasticNetParam=0)\n    model = lr.fit(train)\n    test_m = Metrics()\n    test = model.transform(test).select(\'prediction\', \'label\')\n    #y = str(i)\n    #x = str((test_m.f1(test)))\n    #print(y + x, end = "")\n    test_m.f1(test)\n    test_m.confusion_matrix(test)'